<a href="https://colab.research.google.com/github/Bruno-GSilva/price-prediction/blob/main/model_2/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model 2

### Mount Drive to access data

In [345]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports

In [346]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.functional import F
import torch.optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler

device = 'cpu' #torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Data Loader

In [347]:
class PriceDataset(Dataset):

  def __init__(self, transform=None):

    #Read CSV file
    df = pd.read_csv("/content/drive/MyDrive/data/PETR4.SA.csv", parse_dates=[0], infer_datetime_format=True)
    df_clean = df.dropna().reset_index()

    #Add collumn indicating when there is a time skip > 1 day
    time_skip = [ 0 if index == 0 else 1 if (row['Date'] - df_clean['Date'][index - 1]).days > 1 else 0 for index, row in df_clean.iterrows() ]
    df_clean['Skip dates'] = time_skip

    #Filter usefull collumns
    df_filter = df_clean.filter([ 'Close'])#  'Open', 'High', 'Low', 'Close'])#, 'Skip dates'])
    data = df_filter.values[5000:]

    #Scale input
    #scaler = MinMaxScaler()
    #data = scaler.fit_transform(data)

    #Create samples
    x_samples = []
    y_samples = []
    time_window = 30
    for i in range (time_window, len(data)-time_window):
      scaler = MinMaxScaler()
      x_data = np.array(data[i-time_window : i])
      scaler.fit(x_data)
      scaled_x = scaler.transform(x_data)
      x_samples.append(scaled_x)
      #y_samples.append([1. if data[i+30,3] > data[i, 3] else 0.])
      #y_samples.append([1. if data[i+30,0] > data[i, 0] else 0.])
      y_sample = data[i+30]
      y_scaled = scaler.transform(np.reshape(y_sample, (-1,1)))
      y_samples.append(y_scaled)

    
    x_samples, y_samples = np.array(x_samples), np.array(y_samples)

    #Transform int tensors
    self.x_data = torch.from_numpy(x_samples)
    self.y_data = torch.from_numpy(y_samples)
    self.n_data = y_samples.shape[0]

    #Set transform
    self.transform = transform

  def __getitem__(self, index):
    sample = self.x_data[index], self.y_data[index]
    if self.transform:
            sample = self.transform(sample)
    return sample

  def __len__(self):
    return self.n_data

## Model

In [348]:
class LSTMPrice(nn.Module):

  def __init__(self, num_classes, input_size, hidden_size, num_layers, seq_length):
    super(LSTMPrice, self).__init__()
    self.num_classes = num_classes
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.seq_length = seq_length

    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
    self.fc_1 = nn.Linear(hidden_size, 64)
    self.fc_2 = nn.Linear(64, num_classes)

  def forward(self, x):
    h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #hidden state
    c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size) #internal state
    # Propagate input through LSTM
    output, (hn, cn) = self.lstm(x, (h_0, c_0)) #lstm with input, hidden, and internal state
    #out = torch.reshape(output, (output.size(0), -1))
    hn = hn.view(-1, self.hidden_size) #reshaping the data for Dense layer next
    out = F.relu(hn)
    out = F.relu(self.fc_1(out)) #first Dense
    out = self.fc_2(out) #Final Output

    #Output with sigmoid
    #out = torch.sigmoid(out)

    return out      


## Preparing for trainning

In [349]:
dataset = PriceDataset()

In [350]:
#Hyperparameters
batch_size=4
learning_rate=0.0001
input_size=dataset[0][0].shape[1]
seq_length = dataset[0][0].shape[0]
hidden_size=2
num_epochs=20
num_layers=1
num_classes=dataset[0][1].shape[0]

In [351]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=2)

In [358]:
model = LSTMPrice(num_classes, input_size, hidden_size, num_layers, seq_length)

criterion = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

example_input = dataset[0][0].view(1,30,1)
#print(example_input)
#model(example_input.float())
dataset[0][1]

tensor([[0.6164]], dtype=torch.float64)

In [353]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):  
        if labels.shape[0] == batch_size:
          inputs = inputs.float()
          labels = labels.view(batch_size, 1).float()
          
          # Forward pass
          outputs = model(inputs).float()
          #print(outputs)
          loss = criterion(outputs, labels)
          
          # Backward and optimize
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          
          if (i+1) % 10 == 0:
              print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss}')
              #print(outputs)

Epoch [1/20], Step [10/94], Loss: 1.7129223346710205
Epoch [1/20], Step [20/94], Loss: 8.076231002807617
Epoch [1/20], Step [30/94], Loss: 20.30123519897461
Epoch [1/20], Step [40/94], Loss: 9.732126235961914
Epoch [1/20], Step [50/94], Loss: 2.0270543098449707
Epoch [1/20], Step [60/94], Loss: 10.546252250671387
Epoch [1/20], Step [70/94], Loss: 0.992142379283905
Epoch [1/20], Step [80/94], Loss: 1.590135097503662
Epoch [1/20], Step [90/94], Loss: 1.874682903289795
Epoch [2/20], Step [10/94], Loss: 4.132444381713867
Epoch [2/20], Step [20/94], Loss: 4.543850421905518
Epoch [2/20], Step [30/94], Loss: 0.3235752582550049
Epoch [2/20], Step [40/94], Loss: 1.7787123918533325
Epoch [2/20], Step [50/94], Loss: 5.3101935386657715
Epoch [2/20], Step [60/94], Loss: 2.7233948707580566
Epoch [2/20], Step [70/94], Loss: 0.5187737941741943
Epoch [2/20], Step [80/94], Loss: 6.040611267089844
Epoch [2/20], Step [90/94], Loss: 0.7999733090400696
Epoch [3/20], Step [10/94], Loss: 2.488240957260132
Epo

In [359]:
print(model(example_input.float()))

tensor([[0.0804]], grad_fn=<AddmmBackward>)
